# Lab 1

In [1]:
import numpy as np
from galois import GF, Poly

GF2 = GF(2)
print(GF2.properties)

Galois Field:
  name: GF(2)
  characteristic: 2
  degree: 1
  order: 2
  irreducible_poly: x + 1
  is_primitive_poly: True
  primitive_element: 1


In [2]:
class PolyOps:
    @staticmethod
    def multiply(poly1: Poly, poly2: Poly):
        coeffs1 = poly1.coeffs
        coeffs2 = poly2.coeffs

        result = [0] * (len(coeffs1) + len(coeffs2) - 1)

        for i in range(len(coeffs1)):
            for j in range(len(coeffs2)):
                result[i + j] ^= (coeffs1[i] & coeffs2[j])

        return Poly.Int(int(''.join(map(str, result)), 2), field=GF2)

    @staticmethod
    def mod(poly, mod_poly):
        coeffs = poly.coeffs
        mod_coeffs = mod_poly.coeffs

        mod_degree = len(mod_coeffs) - 1

        for i in range(len(coeffs) - mod_degree):
            if coeffs[i] != 0:
                for j in range(len(mod_coeffs)):
                    coeffs[i + j] ^= mod_coeffs[j]

        remainder = coeffs[-mod_degree:]
        
        return Poly.Int(int(''.join(map(str, remainder)), 2), field=GF2)
    
    @staticmethod
    def add(poly1, poly2):
        coeffs1 = poly1.coeffs[::-1]
        coeffs2 = poly2.coeffs[::-1]

        result = [0] * max(len(coeffs1), len(coeffs2))

        for i in range(len(coeffs1)):
            result[i] ^= coeffs1[i]
        for i in range(len(coeffs2)):
            result[i] ^= coeffs2[i]

        return Poly.Int(int(''.join(map(str, result[::-1])), 2), field=GF2)


In [3]:
bin_g = "1011"
bin_m = "1010"
bin_e = "1100101" # will be justified for len (k + r)

## Encode

In [4]:
g = Poly.Int(int(bin_g, 2), field=GF2)
r = g.degree

print('bin(g) =', bin_g)
print('g(x) =', g)
print('r =', r)

bin(g) = 1011
g(x) = x^3 + x + 1
r = 3


In [5]:
m = Poly.Int(int(bin_m, 2), field=GF2)
deg_m = m.degree
k = len(m)

print('bin(m) =', bin_m)
print('m(x) = ', m)
print('deg(m) <= k - 1 :', deg_m <= k - 1)

bin(m) = 1010
m(x) =  x^3 + x
deg(m) <= k - 1 : True


In [6]:
x_pow_r = Poly.Degrees([r], field=GF2)

# c = (m * x_pow_r) % g
c = PolyOps.mod(PolyOps.multiply(m, x_pow_r), g)
deg_c = c.degree

print('x^r = ', x_pow_r)
print('c(x) =', c)
print('deg(c) <= r - 1 :', deg_c <= r - 1)

x^r =  x^3
c(x) = x + 1
deg(c) <= r - 1 : True


In [7]:
# a = (m * x_pow_r) + c
a = PolyOps.multiply(m, x_pow_r) + c

n = k + r
bin_a = bin(a)[2:].rjust(n, '0')

print('a(x) =', a)
print('bin(a) =', bin_a)
print('a(x) mod g(x) = 0 :', a % g == 0)

a(x) = x^6 + x^4 + x + 1
bin(a) = 1010011
a(x) mod g(x) = 0 : True


In [8]:
def bin_str_xor(a: str, b: str) -> str:
    max_len = max(len(a), len(b))
    result = int(a, 2) ^ int(b, 2)
    return bin(result)[2:].rjust(max_len, '0')

In [9]:
bin_e = bin_e.rjust(n, '0')
bin_b = bin_str_xor(bin_a, bin_e)

print('bin(a) =', bin_a)
print('bin(e) =', bin_e)
print('bin(b) =', bin_b)

bin(a) = 1010011
bin(e) = 1100101
bin(b) = 0110110


## Decode

In [10]:
b = Poly.Int(int(bin_b, 2), field=GF2)

print('b(x) =', b)

b(x) = x^5 + x^4 + x^2 + x


In [11]:
# s = b % g
s = PolyOps.mod(b, g)
E = s != 0

print('s(x) =', s)
print('s(x) = 0 :', s == 0)
print('E =', E)

s(x) = x^2 + x + 1
s(x) = 0 : False
E = True


### Dop B

In [12]:
def error_channel_model(bin_g, bin_m, bin_e):
    g = Poly.Int(int(bin_g, 2), field=GF2)
    r = g.degree
    m = Poly.Int(int(bin_m, 2), field=GF2)
    k = len(m)
    x_pow_r = Poly.Degrees([r], field=GF2)
    c = PolyOps.mod(PolyOps.multiply(m, x_pow_r), g)
    a = PolyOps.multiply(m, x_pow_r) + c
    n = k + r
    bin_a = bin(a)[2:].rjust(n, '0')
    bin_e = bin_e.rjust(n, '0')
    bin_b = bin_str_xor(bin_a, bin_e)
    b = Poly.Int(int(bin_b, 2), field=GF2)
    s = PolyOps.mod(b, g)
    E = s != 0
    return E


print("Error found:", error_channel_model(bin_g, bin_m, bin_e))

Error found: True


In [28]:
l, d = 4, 3
n = l + d

e_vectors = [
    bin(i)[2:].rjust(n, '0') for i in range(2**n)
    if bin(i).count('1') <= (d - 1) and bin(i).count('1') > 0
]

bin_a = bin_a.rjust(n, '0')
for bin_e in e_vectors:
    if not error_channel_model(bin_g, bin_m, bin_e):
        print(f'Вектор ошибок {bin_e} НЕ обнаруживает E')

In [33]:
def calc_for_custom_l(l, bin_a):
    d = 3
    n = l + d

    e_vectors = [
        bin(i)[2:].rjust(n, '0') for i in range(2**n)
        if bin(i).count('1') <= (d - 1) and bin(i).count('1') > 0
    ]

    bin_a = bin_a.rjust(n, '0')
    for bin_e in e_vectors:
        if not error_channel_model(bin_g, bin_m, bin_e):
            print(f'Вектор ошибок {bin_e} НЕ обнаруживает E')

In [46]:
calc_for_custom_l(20, bin_a)

Вектор ошибок 00000000000000010000001 НЕ обнаруживает E
Вектор ошибок 00000000000000100000010 НЕ обнаруживает E
Вектор ошибок 00000000000001000000100 НЕ обнаруживает E
Вектор ошибок 00000000000010000001000 НЕ обнаруживает E
Вектор ошибок 00000000000100000010000 НЕ обнаруживает E
Вектор ошибок 00000000001000000100000 НЕ обнаруживает E
Вектор ошибок 00000000010000001000000 НЕ обнаруживает E
Вектор ошибок 00000000100000000000001 НЕ обнаруживает E
Вектор ошибок 00000000100000010000000 НЕ обнаруживает E
Вектор ошибок 00000001000000000000010 НЕ обнаруживает E
Вектор ошибок 00000001000000100000000 НЕ обнаруживает E
Вектор ошибок 00000010000000000000100 НЕ обнаруживает E
Вектор ошибок 00000010000001000000000 НЕ обнаруживает E
Вектор ошибок 00000100000000000001000 НЕ обнаруживает E
Вектор ошибок 00000100000010000000000 НЕ обнаруживает E
Вектор ошибок 00001000000000000010000 НЕ обнаруживает E
Вектор ошибок 00001000000100000000000 НЕ обнаруживает E
Вектор ошибок 00010000000000000100000 НЕ обнаруж

In [14]:
l = 4
d = 3

e_vectors = [
    bin(i)[2:].rjust(l + d, '0') for i in range(2**n)
    if bin(i).count('1') <= (d - 1)
]

for e in e_vectors:
    bin_e = e.rjust(n, '0')
    bin_b = bin_str_xor(bin_a, bin_e)
    b = Poly.Int(int(bin_b, 2), field=GF2)
    s = b % g
    E = s != 0
    if not E:
        print(f'Вектор ошибок {e} не обнаруживает ошибки.')


Вектор ошибок 0000000 не обнаруживает ошибки.
Вектор ошибок 10000001 не обнаруживает ошибки.
Вектор ошибок 100000010 не обнаруживает ошибки.
Вектор ошибок 1000000100 не обнаруживает ошибки.
